In [20]:
import kagglehub
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import math
import numpy as np

Access the appendix of this paper (https://www.irjet.net/archives/V3/i4/IRJET-V3I4213.pdf) to read more about the meaning of each columns.

In [21]:
# Download latest version
path = kagglehub.dataset_download("jpacse/datasets-for-churn-telecom")

# List files in the directory to find the CSV
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]

if not csv_files:
    raise FileNotFoundError(f"No CSV files found in {path}")

# Construct path to the CSV file
csv_path = os.path.join(path, csv_files[0])

# Load the CSV into a DataFrame
df = pd.read_csv(csv_path)

In [22]:
df.head().style.set_properties()

,CustomerID,Churn,MonthlyRevenue,MonthlyMinutes,TotalRecurringCharge,DirectorAssistedCalls,OverageMinutes,RoamingCalls,PercChangeMinutes,PercChangeRevenues,DroppedCalls,BlockedCalls,UnansweredCalls,CustomerCareCalls,ThreewayCalls,ReceivedCalls,OutboundCalls,InboundCalls,PeakCallsInOut,OffPeakCallsInOut,DroppedBlockedCalls,CallForwardingCalls,CallWaitingCalls,MonthsInService,UniqueSubs,ActiveSubs,ServiceArea,Handsets,HandsetModels,CurrentEquipmentDays,AgeHH1,AgeHH2,ChildrenInHH,HandsetRefurbished,HandsetWebCapable,TruckOwner,RVOwner,Homeownership,BuysViaMailOrder,RespondsToMailOffers,OptOutMailings,NonUSTravel,OwnsComputer,HasCreditCard,RetentionCalls,RetentionOffersAccepted,NewCellphoneUser,NotNewCellphoneUser,ReferralsMadeBySubscriber,IncomeGroup,OwnsMotorcycle,AdjustmentsToCreditRating,HandsetPrice,MadeCallToRetentionTeam,CreditRating,PrizmCode,Occupation,MaritalStatus
0,3000002,Yes,24.000000,219.000000,22.000000,0.250000,0.000000,0.000000,-157.000000,-19.000000,0.700000,0.700000,6.300000,0.000000,0.000000,97.200000,0.000000,0.000000,58.000000,24.000000,1.300000,0.000000,0.300000,61,2,1,SEAPOR503,2.000000,2.000000,361.000000,62.000000,0.000000,No,No,Yes,No,No,Known,Yes,Yes,No,No,Yes,Yes,1,0,No,No,0,4,No,0,30,Yes,1-Highest,Suburban,Professional,No
1,3000010,Yes,16.990000,10.000000,17.000000,0.000000,0.000000,0.000000,-4.000000,0.000000,0.300000,0.000000,2.700000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1.000000,0.300000,0.000000,0.000000,58,1,1,PITHOM412,2.000000,1.000000,1504.000000,40.000000,42.000000,Yes,No,No,No,No,Known,Yes,Yes,No,No,Yes,Yes,0,0,Yes,No,0,5,No,0,30,No,4-Medium,Suburban,Professional,Yes
2,3000014,No,38.000000,8.000000,38.000000,0.000000,0.000000,0.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,0.300000,0.000000,1.300000,3.700000,0.000000,0.000000,0.000000,60,1,1,MILMIL414,1.000000,1.000000,1812.000000,26.000000,26.000000,Yes,No,No,No,No,Unknown,No,No,No,No,No,Yes,0,0,Yes,No,0,6,No,0,Unknown,No,3-Good,Town,Crafts,Yes
3,3000022,No,82.280000,1312.000000,75.000000,1.240000,0.000000,0.000000,157.000000,8.100000,52.000000,7.700000,76.000000,4.300000,1.300000,200.300000,370.300000,147.000000,555.700000,303.700000,59.700000,0.000000,22.700000,59,2,2,PITHOM412,9.000000,4.000000,458.000000,30.000000,0.000000,No,No,Yes,No,No,Known,Yes,Yes,No,No,No,Yes,0,0,Yes,No,0,6,No,0,10,No,4-Medium,Other,Other,No
4,3000026,Yes,17.140000,0.000000,17.000000,0.000000,0.000000,0.000000,0.000000,-0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,53,2,2,OKCTUL918,4.000000,3.000000,852.000000,46.000000,54.000000,No,No,No,No,No,Known,Yes,Yes,No,No,Yes,Yes,0,0,No,Yes,0,9,No,1,10,No,1-Highest,Other,Professional,Yes


In [23]:
# look through the datatype of each column

df.dtypes

CustomerID                     int64
Churn                         object
MonthlyRevenue               float64
MonthlyMinutes               float64
TotalRecurringCharge         float64
DirectorAssistedCalls        float64
OverageMinutes               float64
RoamingCalls                 float64
PercChangeMinutes            float64
PercChangeRevenues           float64
DroppedCalls                 float64
BlockedCalls                 float64
UnansweredCalls              float64
CustomerCareCalls            float64
ThreewayCalls                float64
ReceivedCalls                float64
OutboundCalls                float64
InboundCalls                 float64
PeakCallsInOut               float64
OffPeakCallsInOut            float64
DroppedBlockedCalls          float64
CallForwardingCalls          float64
CallWaitingCalls             float64
MonthsInService                int64
UniqueSubs                     int64
ActiveSubs                     int64
ServiceArea                   object
H

# Data Quality Assessment and Preprocessing

Note: While null datas are often imputed in the preprocessing stage, this may lead to my numerical and graphical in EDA to be flawed. Therefore, I decided to transfer null imputations to the end of my EDA .ipynb. 

In [24]:
# Check for duplicated CustomerID 

print(df['CustomerID'].duplicated().any())

False


In [25]:
unique_vals = {
    col: df[col].unique()
    for col in df.select_dtypes(include=['object', 'category']).columns
}

# print(unique_vals)

# MaritalStatus and HandsetPrice contains 'Unknown' => convert them to nan
df.replace('Unknown', np.nan, inplace=True)

In [26]:
print(f"Column(s) where there are only 1 unique values: {[col for col in df if df[col].nunique() == 1]}")

# recheck
print(df['Homeownership'].value_counts())

# The value for Homeownership is always 0 so they are not meaningful 
# Drop the column 'Homeownership' from df

df = df.drop('Homeownership', axis=1)

Column(s) where there are only 1 unique values: ['Homeownership']
Homeownership
Known    33987
Name: count, dtype: int64


In [27]:
print(f"Number of unique values for ServiceArea: {df['ServiceArea'].nunique()}")

# There are too many ServiceArea so we will convert them to just include cities
df['ServiceCity'] = df['ServiceArea'].str[:3]

print(f"Number of unique values for ServiceCity: {df['ServiceCity'].nunique()}")


Number of unique values for ServiceArea: 747
Number of unique values for ServiceCity: 57


In [28]:
# Count total rows in the dataframe
print(f"Total rows in dataframe: {len(df)}") 

# Count the percentage of rows where AgeHH1 == 0
print(f"% of rows in dataframe where AgeHH1 is 0: {round((df['AgeHH1'] == 0).sum() / len(df) * 100, 2)}") 

# Count the percentage of rows where AgeHH1 == 0 AND ChildrenInHH == "No"
print(f"% of rows in dataframe where AgeHH1 is 0 and ChildrenInHH is 'No': {round(((df['AgeHH1'] == 0) & (df['ChildrenInHH'] == 'No')).sum() / len(df) * 100, 2)}") 

# Note: There are households where age of head household is 0 but there are children.
# Secondary sources did not explain why AgeHH1 is 0 so we will assume that these are meant to be null. 
df.loc[df['AgeHH1'] == 0, 'AgeHH1'] = np.nan
print("AgeHH1 = 0 finished converting to null")

Total rows in dataframe: 51047
% of rows in dataframe where AgeHH1 is 0: 27.26
% of rows in dataframe where AgeHH1 is 0 and ChildrenInHH is 'No': 27.03
AgeHH1 = 0 finished converting to null


In [29]:
# convert income group to categorical

df['IncomeGroup'] = df['IncomeGroup'].astype('category')

In [30]:
# (Umayaparvathi and Iyakutti 1070) mentioned that IncomeGroup = 0 indicates missing data
# So, convert IncomeGroup = 0 to nan

df['IncomeGroup'] = df['IncomeGroup'].copy()
df.loc[df['IncomeGroup'] == 0, 'IncomeGroup'] = np.nan

In [31]:
# Convert certain numerical column to categorical

to_be_converted = ['HandsetWebCapable','TruckOwner', 'RVOwner', 'ChildrenInHH']

# Convert 1s to True and 0s to False
for col in to_be_converted:
    df.loc[df[col] == 0, col] = "False"
    df.loc[df[col] == 1, col] = "True"

# Check
for col in to_be_converted:
    print(df[col].value_counts())

HandsetWebCapable
Yes    46046
No      5001
Name: count, dtype: int64
TruckOwner
No     41524
Yes     9523
Name: count, dtype: int64
RVOwner
No     46894
Yes     4153
Name: count, dtype: int64
ChildrenInHH
No     38679
Yes    12368
Name: count, dtype: int64


In [32]:
for col in df:
    df.loc[df[col] == 'Unknown', col] = np.nan    

In [34]:
def columns_with_mixed_types(df):
    mixed_cols = []
    for col in df.columns:
        # Get unique types in the column (excluding NaNs)
        types = df[col].dropna().map(type).unique()
        if len(types) > 1:
            mixed_cols.append((col, types))
    return mixed_cols

mixed_columns = columns_with_mixed_types(df)
print("Columns with mixed types and their types:")
for col, types in mixed_columns:
    print(f"{col}: {types}")

Columns with mixed types and their types:


In [35]:
# Log-transform columns to address skewness > 1

column_list = df.columns.drop('CustomerID').tolist()

for col_name in column_list:
    if pd.api.types.is_numeric_dtype(df[col_name]):  
        # filter for columns where skewness > 1 and there is no negative values
        if df[col_name].skew() > 1 and df[col_name].min() >= 0: 
            df[col_name] = np.log1p(df[col_name])  # handles 0 safely: log(1 + x)
            print(f"Log-transformed: {col_name}")

Log-transformed: MonthlyMinutes
Log-transformed: DirectorAssistedCalls
Log-transformed: OverageMinutes
Log-transformed: RoamingCalls
Log-transformed: DroppedCalls
Log-transformed: BlockedCalls
Log-transformed: UnansweredCalls
Log-transformed: CustomerCareCalls
Log-transformed: ThreewayCalls
Log-transformed: ReceivedCalls
Log-transformed: OutboundCalls
Log-transformed: InboundCalls
Log-transformed: PeakCallsInOut
Log-transformed: OffPeakCallsInOut
Log-transformed: DroppedBlockedCalls
Log-transformed: CallForwardingCalls
Log-transformed: CallWaitingCalls
Log-transformed: MonthsInService
Log-transformed: UniqueSubs
Log-transformed: ActiveSubs
Log-transformed: Handsets
Log-transformed: HandsetModels
Log-transformed: RetentionCalls
Log-transformed: RetentionOffersAccepted
Log-transformed: ReferralsMadeBySubscriber
Log-transformed: AdjustmentsToCreditRating


In [36]:
df.to_csv('cleaned_data.csv', index=False)